In [ ]:
%matplotlib inline
#imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import silhouette_score

## Create our pipeline and define the classifier we plan to use for stocks within any of the 3 indices

In [ ]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler())
    ]
)
kmeans_kwargs = { "init": "k-means++", "n_init": 10, "max_iter": 300, "random_state": 42 }
clf = Pipeline([
    (
        "kmeans",
     KMeans(
         **kmeans_kwargs
     ),
    ),
])
pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clf)
    ]
)

In [ ]:
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
nasdaq100_url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
russell1000_url = 'https://en.wikipedia.org/wiki/Russell_1000_Index'
indices = {
    'sp500': {'index_url': sp500_url, 'table_num': 0, 'index_name': 'sp500'},
    'nq100': {'index_url': nasdaq100_url, 'table_num': 3, 'index_name': 'nasdaq100'},
    'rs1000': {'index_url': russell1000_url, 'table_num': 2, 'index_name': 'russell1000'}
}
engine = create_engine('sqlite:///indices_data', echo=False)

In [ ]:
#read in the url and scrape ticker data
def get_stocks_data(index_url, table_num, index_name):
    """
    @param index_url: link to index (currently using Wikipedia)
    @param table_num: table number on webpage since parsing HTML
    @return: pd dataframe with close price data for all stocks in the index    
    """
    # if we already have data no need to fetch, ret DF with pricing data for stocks in index 
    if index_name in set(engine.table_names()):
        return pd.read_sql_table(table_name=index_name, con=engine, index_col='Date')
    # otherwise, parse html, get prices for each ticker, etc.
    data_table = pd.read_html(index_url)
    # nq100 and rus1000 both have ticker in table instead of symbol.  
    col_name = 'Symbol' if index_name == 'sp500' else 'Ticker'
    tickers = data_table[table_num][col_name].values
    prices_list = []
    for idx, ticker in enumerate(tickers):
        try:
#             print(f"Working with ticker...:{ticker} and {len(tickers) - idx} left to go.")
            # refactor (nothing really to change except merge/join v. concat)
            prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Adj Close']
            ticker_prices = pd.DataFrame(prices)
            ticker_prices.columns = [ticker]
            prices_list.append(ticker_prices)
        except:
#             print(f"Error with retrieving data for: {ticker}")
            pass
        all_tickers_prices = pd.concat(prices_list, axis=1)
#         print(f"Added {ticker} to dataframe.")
    # sort alphabetically by ticker
    all_tickers_prices.sort_index(inplace=True)
    # cache prices in DB for future use
    all_tickers_prices.to_sql(f'{index_name}', con=engine, if_exists='replace')
    print(f'Finished {index_name}')
    return all_tickers_prices


#Calculate average annualized percentage return and volatilities over a theoretical one year period
def calc_returns_vol(index_stock_prices, index_name):    
    returns = index_stock_prices.pct_change().mean() * 252
    volatility = index_stock_prices.pct_change().std() * sqrt(252)
    ret_vol = pd.DataFrame({'Returns': returns, 'Volatility': volatility})
    return ret_vol


# fit Kmeans, get preprocessed data to use for plotting later. 
def kmeans_pipe_process(ret_vol):
    pipe.fit(ret_vol[['Returns', 'Volatility']])
    preprocessed_data = pipe['preprocessor'].transform(ret_vol[['Returns', 'Volatility']])
#     print(f'silhouette score is {silhouette_score(preprocessed_data, predicted_labels)}')
    # first col is normalized returns, second is volatility
    scaled_ret_vol = pd.DataFrame({'Returns': preprocessed_data[:, 0], 'Volatility': preprocessed_data[:, 1]})
#     print(scaled_ret_vol.head())
    # can't return tuple since kmeans isn't iterable. 
    return {'kmeans': pipe['clusterer']['kmeans'], 'scaled': scaled_ret_vol}

In [ ]:
# prices_df.head()

In [ ]:
from kneed import KneeLocator

def plot_elbow_curve(distortions):
    fig = plt.figure(figsize=(15, 5))
    plt.style.use('fivethirtyeight')
    plt.plot(range(1, len(distortions) + 1), distortions)
    plt.xlabel('Number of Clusters')
    plt.ylabel("SSE")
    plt.title('Elbow curve')
    plt.grid(True)
    plt.show()


def find_n_clusters(scaled_ret_vol, kmeans):
    X = scaled_ret_vol[['Returns', 'Volatility']]
    distortions = []
    # 11 sectors in SP500, we are looking for optimal k for kmeans
    for k in range(1, 12):
        kmeans.n_clusters = k
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)
#         print(f"Num clusters: {kmeans.n_clusters}")
    # display elbow curve
    plot_elbow_curve(distortions)
    kl = KneeLocator(range(1, len(distortions) + 1), distortions, S=1.0, curve="convex", direction="decreasing")
    # kmeans_kwargs['n_clusters'] = kl.elbow
    kmeans.n_clusters = kl.elbow
#     print(f'num_clusters is: {kmeans.n_clusters}')
    return kmeans.fit(scaled_ret_vol[['Returns', 'Volatility']])
    
    
def disp_clusters_show_outliers(scaled_ret_vol, kmeans, index_name):
    fig = plt.figure(figsize=(20, 20))
#     colors = ['red', 'blue', 'purple', 'yellow']
    sns.scatterplot(x='Returns', y='Volatility', hue=kmeans.labels_, data=scaled_ret_vol, palette="deep", s=100)
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.title(f"{index_name} Stocks Clustered into {kmeans.n_clusters} groups")
    centers = kmeans.cluster_centers_
#     print(centers)
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=400, alpha=0.5);
    
    
def display_initial_results(scaled_ret_vol, kmeans, index_name):
    find_n_clusters(scaled_ret_vol, kmeans)
    disp_clusters_show_outliers(scaled_ret_vol, kmeans, index_name)

In [ ]:
def run_kmeans_all_indices(indices_data):
    preprocessed_data_indices = {}
    for index in indices_data:
        index_ret_vol = calc_returns_vol(
            # unpack args from dict into kwargs for get_stocks_data
            index_stock_prices=get_stocks_data(**indices_data[index]), 
            index_name=indices[index]['index_name']
        )
#         print('before kmeans processing', index_ret_vol.head())
        clf_preprocessed_data = kmeans_pipe_process(index_ret_vol)
        preprocessed_data_indices[indices[index]['index_name']] = clf_preprocessed_data
        kmeans_index, scaled_ret_vol = clf_preprocessed_data['kmeans'], clf_preprocessed_data['scaled']
        display_initial_results(scaled_ret_vol, kmeans_index, indices_data[index]['index_name'])
        
    return preprocessed_data_indices

### We see above from the plot that the "elbow" i.e., optimal number of clusters "k" is 4 but we can also find it programmatically

In [ ]:
# remove outliers from data set to get a more clear kmeans clustering
def drop_outliers(ret_vol, ret_threshold, vol_threshold):
    rm_outliers = ret_vol[(ret_vol['Returns'] < ret_threshold) & (ret_vol['Volatility'] < vol_threshold)]
    return rm_outliers
    # data = np.asarray([np.asarray(returns['Returns']),np.asarray(returns['Volatility'])]).T

def plot_final_kmeans(index_name, kmeans, *args):
    """
    @param index_name: name of stock index
    @param kmeans: kmeans clf for index
    @param *args: args to be passed to subsequent functions (df, returns/vol thresholds)
    """
    # unpack variable args tuple needed for pos. args drop_outliers
    returns_rm_outliers = drop_outliers(*args)
    predicted_labels = find_n_clusters(returns_rm_outliers, kmeans).labels_
#     predicted_labels = kmeans.fit(returns_rm_outliers[['Returns', 'Volatility']]).labels_
    returns_rm_outliers['predicted_label'] = predicted_labels
    centers = kmeans.cluster_centers_
#     print(centers)
    fig = plt.figure(figsize=(20, 20))
#     colors = ['red', 'blue', 'purple', 'green']
    sns.scatterplot(x='Returns', y='Volatility', hue='predicted_label', data=returns_rm_outliers, palette='deep', s=100)
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.title(f"{index_name} Stocks Clustered into {kmeans.n_clusters} groups")
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=400, alpha=0.5);
    plt.savefig(f"C:\\Users\\joshu\\PycharmProjects\\finance-project\\website\\static\\images\\{index_name}.png")

## Finally, we plot the data after running KMeans with the outliers removed 

In [ ]:
prepr_data_indices = run_kmeans_all_indices(indices)
# print(prepr_data_indices)

## From the charts above, we see the outliers for each index after preprocessing are in the range of: 
- S&P500: Returns $\geq$ 0.40 and Volatility $\geq$ 0.6 
- Nasdaq 100: Returns $\geq$ 0.6 and Volatility $\geq$ 0.6
- Russell 1000: Returns $\geq$ 0.2 and Volatility $\geq$ 0.1

## Plotting final elbow charts with new number of clusters and filtered data:

In [ ]:
prepr_data_indices['sp500']['ret_threshold'], prepr_data_indices['sp500']['vol_threshold'] = 0.4, 0.6
prepr_data_indices['nasdaq100']['ret_threshold'], prepr_data_indices['nasdaq100']['vol_threshold'] = 0.6, 0.6
prepr_data_indices['russell1000']['ret_threshold'], prepr_data_indices['russell1000']['vol_threshold'] = 0.2, 0.1
for index in prepr_data_indices:
    outliers_dropped = plot_final_kmeans(index, prepr_data_indices[index]['kmeans'], prepr_data_indices[index]['scaled'], prepr_data_indices[index]['ret_threshold'], prepr_data_indices[index]['vol_threshold'])